# Fabric Date Table Generator (T-SQL Approach)

This notebook generates a dynamic date table using T-SQL, suitable for Microsoft Fabric Lakehouse, Synapse, or Power BI Direct Lake scenarios.

## Parameters

Define the parameters for the date table generation. Adjust as needed:
- **StartDate**: First date in the table (YYYY-MM-DD)
- **EndDate**: Last date in the table (YYYY-MM-DD)
- **FiscalYearStartMonth**: Month (1-12) when the fiscal year starts
- **HolidayDates**: Comma-separated list of holiday dates (YYYY-MM-DD) to flag in the table

In [ ]:
-- Parameters
-- Replace scalar variables with inline values for Microsoft Fabric compatibility
-- StartDate and EndDate
WITH DateParameters AS (
    SELECT
        CAST('2015-01-01' AS DATE) AS StartDate,
        CAST('2030-12-31' AS DATE) AS EndDate,
        7 AS FiscalYearStartMonth,
        '2025-01-01,2025-12-25' AS HolidayDates
)
SELECT * FROM DateParameters;

## Generate Date Table

Use a CTAS (CREATE TABLE AS SELECT) pattern to generate the date range.

In [ ]:
-- Generate date range using a numbers table approach for Lakehouse
WITH Numbers AS (
    SELECT ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS n
    FROM (VALUES (1), (1), (1), (1), (1), (1), (1), (1), (1), (1)) AS t1(n)
         CROSS JOIN (VALUES (1), (1), (1), (1), (1), (1), (1), (1), (1), (1)) AS t2(n)
         CROSS JOIN (VALUES (1), (1), (1), (1), (1), (1), (1), (1), (1), (1)) AS t3(n)
)
SELECT TOP (DATEDIFF(DAY, CAST('2015-01-01' AS DATE), CAST('2030-12-31' AS DATE)) + 1)
    DATEADD(DAY, n - 1, CAST('2015-01-01' AS DATE)) AS [Date]
INTO FabricLakehouse.DateRange
FROM Numbers;

-- Display the generated data
SELECT TOP 10 * FROM FabricLakehouse.DateRange;

DatawarehouseExecuteSqlError: Must declare the scalar variable "@StartDate".
Must declare the scalar variable "@EndDate".
Query hint 'unknown hint' is not allowed. Data warehouse trace id: e7678417-9e17-4bfa-a90f-0b6e251fa4a3

## Add Calendar and Fiscal Columns

Add columns for year, month, quarter, fiscal year, fiscal month, and fiscal quarter.

In [ ]:
-- Add calendar and fiscal columns
CREATE TABLE DateTableEnriched AS
SELECT
    [Date],
    YEAR([Date]) AS [Year],
    MONTH([Date]) AS [Month],
    DATENAME(MONTH, [Date]) AS [MonthName],
    DAY([Date]) AS [Day],
    DATEPART(QUARTER, [Date]) AS [Quarter],
    DATEPART(WEEK, [Date]) AS [WeekOfYear],
    DATENAME(WEEKDAY, [Date]) AS [DayOfWeekName],
    DATEPART(WEEKDAY, [Date]) AS [DayOfWeek],
    -- Fiscal Year logic
    CASE WHEN MONTH([Date]) >= @FiscalYearStartMonth THEN YEAR([Date]) + 1 ELSE YEAR([Date]) END AS [FiscalYear],
    ((MONTH([Date]) - @FiscalYearStartMonth + 12) % 12 + 1) AS [FiscalMonth],
    (( ((MONTH([Date]) - @FiscalYearStartMonth + 12) % 12 ) / 3 ) + 1) AS [FiscalQuarter]
FROM DateRange;

## Flag Holidays and Current Dates

Add columns to flag holidays and current day/month/year.

In [ ]:
-- Split holiday dates into a table variable
DECLARE @HolidayTable TABLE (HolidayDate DATE);
INSERT INTO @HolidayTable (HolidayDate)
SELECT value FROM STRING_SPLIT(@HolidayDates, ',');

-- Add holiday and current date flags
CREATE TABLE DateTableFinal AS
SELECT *,
    CASE WHEN [Date] IN (SELECT HolidayDate FROM @HolidayTable) THEN 1 ELSE 0 END AS [IsHoliday],
    CASE WHEN [Date] = CAST(GETDATE() AS DATE) THEN 1 ELSE 0 END AS [IsCurrentDay],
    CASE WHEN [Year] = YEAR(GETDATE()) AND [Month] = MONTH(GETDATE()) THEN 1 ELSE 0 END AS [IsCurrentMonth],
    CASE WHEN [Year] = YEAR(GETDATE()) THEN 1 ELSE 0 END AS [IsCurrentYear]
FROM DateTableEnriched;

## Write Date Table to SQL Database

Persist the final date table to a permanent table. Adjust the schema/table name as needed.

In [ ]:
-- Write to permanent table (change schema/table name as needed)
CREATE TABLE dbo.DateTable AS
SELECT * FROM DateTableFinal;

## Testing & Validation

Validate fiscal year logic and print sample output.

In [ ]:
-- Test: Fiscal year boundary
SELECT TOP 5 * FROM dbo.DateTable WHERE MONTH([Date]) = @FiscalYearStartMonth ORDER BY [Date];

-- Show first and last few rows
SELECT TOP 3 * FROM dbo.DateTable ORDER BY [Date];
SELECT TOP 3 * FROM dbo.DateTable ORDER BY [Date] DESC;